In [85]:
### importing the required modules

import tensorflow as tf
import pandas as pd
from tensorflow import keras

import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import utils
import csv
import pandas as pd

In [ ]:
# get data files
#loading the data

!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [86]:
## converting to dataFrame and Preprocessing

train_df = pd.read_csv(train_file_path, sep='\t', names=['Classification', 'Text'])
test_df = pd.read_csv(test_file_path, sep='\t', names=['Classification', 'Text'])

In [91]:
## Mapping the data

x_train = train_df.drop('Classification', axis=1)
y_train = train_df.drop('Text', axis=1).astype('category')

x_test = test_df.drop('Classification', axis=1)
y_test = test_df.drop('Text', axis=1).astype('category')

In [92]:
### encoding the spam and ham into [0,1]

y_train =  y_train['Classification'].cat.codes
y_test = y_test['Classification'].cat.codes

In [93]:
## iterate over column (text) and encode raw text
VOCAB_SIZE = 1000
for i in x_train:
  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
      max_tokens=VOCAB_SIZE)
  encoder.adapt(np.array(x_train[i]))

In [94]:
for j in x_test:
  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
      max_tokens=VOCAB_SIZE)
  encoder.adapt(np.array(x_test[j]))

In [95]:
vocab = np.array(encoder.get_vocabulary())

In [100]:
### encode the text into indexes (y_train)
for i in x_train:
  data_train = x_train[i]
  encoded_x_train = encoder(data_train)[:3].numpy()


In [97]:
### encode the text into indexes (y_train)
for i in x_test:
  data_test = x_test[i]
  encoded_x_test = encoder(data_test)[:3].numpy()

In [101]:
for n in range(3):
  print("Original: ", data_train[n])
  print("Round-trip: ", " ".join(vocab[encoded_x_train[n]]))
  print()

Original:  ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.
Round-trip:  [UNK] [UNK] [UNK] a bad [UNK] about u [UNK] i dont like u right now i didnt know anything about [UNK] night but i guess im up for it                                                                                                                                              

Original:  you can never do nothing
Round-trip:  you can never do nothing                                                                                                                                                                      

Original:  now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?
Round-trip:  now u [UNK] like [UNK] [UNK] boy [UNK] i is [UNK] on da bus [UNK] has u [UNK] 4 [UNK] dis [UNK]                                                                             

In [102]:
#### creat the model

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [103]:
# predict on a sample text without padding.

sample_text = ("how are you doing today?")
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[-0.01117236]


In [104]:
# predict on a sample text with padding
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[-0.01117236]


In [105]:
###compiling the model

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [107]:
model.fit(x=x_train,y=y_train,epochs=10,validation_data=(x_test, y_test), verbose=1)

Epoch 1/10
131/131 [==============================] - 3s 20ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 0.1213 - val_accuracy: 0.9849
Epoch 2/10
131/131 [==============================] - 3s 22ms/step - loss: 0.0018 - accuracy: 0.9998 - val_loss: 0.1245 - val_accuracy: 0.9849
Epoch 3/10
131/131 [==============================] - 3s 20ms/step - loss: 0.0018 - accuracy: 0.9998 - val_loss: 0.1254 - val_accuracy: 0.9849
Epoch 4/10
131/131 [==============================] - 2s 19ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.1299 - val_accuracy: 0.9849
Epoch 5/10
131/131 [==============================] - 2s 19ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.1303 - val_accuracy: 0.9849
Epoch 6/10
131/131 [==============================] - 3s 19ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.1335 - val_accuracy: 0.9849
Epoch 7/10
131/131 [==============================] - 2s 19ms/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.1334 - val_accuracy: 0.9849

In [122]:
predictions = model.predict_classes(x_test)
print(x_test[7:8])
print("probability",y_test[7:8])
print("prediction was",predictions[7])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


                               Text
7  yes. it's all innocent fun. o:-)
probability 7    0
dtype: int8
prediction was [0]
